In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
import os
from collections import Counter
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
def seed_assign(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
class Qurioga_Spike(Dataset):
    def __init__(self, root = '/data2/spike_sorting/quiroga/', train = True):
        # root = '/data2/spike_sorting/quiroga/'
        if train == True:
            root = root + 'train/'
            # 데이터 로드
            train_data_loaded = torch.load(root + 'data.pt')
            train_label_loaded = torch.load(root + 'label.pt')

            # 데이터 타입 변환
            self.data = train_data_loaded.to(torch.float32)  # train_data를 float32로 변환
            self.labels = train_label_loaded.to(torch.long)  # train_label을 long으로 변환
        else:
            root = root + 'test/'
            # 데이터 로드
            test_data_loaded = torch.load(root + 'data.pt')
            test_label_loaded = torch.load(root + 'label.pt')

            # 데이터 타입 변환
            self.data = test_data_loaded.to(torch.float32)
            self.labels = test_label_loaded.to(torch.long) 

        self.class_num = max(self.labels).item()

    def __len__(self):
        return len(self.data)  # 데이터의 크기 반환

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]  # 주어진 인덱스에 해당하는 데이터와 레이블 반환

# 데이터셋 준비
# train_dataset = Qurioga_Spike(root = '/data2/spike_sorting/quiroga/', train = True)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [4]:
# DNN 모델 정의
class SimpleDNN(nn.Module):
    def __init__(self):
        super(SimpleDNN, self).__init__()

        # self.fc1 = nn.Linear(50, 128)  # 첫 번째 레이어, 입력 크기 50
        # self.fc2 = nn.Linear(128, 64)  # 두 번째 레이어
        # self.fc3 = nn.Linear(64, 12)   # 출력 레이어, 클래스 개수 12개

        self.fc1 = nn.Linear(50, 256)  # 첫 번째 레이어, 입력 크기 50
        self.fc2 = nn.Linear(256, 12)  # 두 번째 레이어

    def forward(self, x):
        # x = torch.relu(self.fc1(x))  # ReLU 활성화 함수
        # x = torch.relu(self.fc2(x))  # ReLU 활성화 함수
        # x = self.fc3(x)              # 최종 출력
        # return x
    
        x = torch.relu(self.fc1(x))  # ReLU 활성화 함수
        x = self.fc2(x)              # ReLU 활성화 함수
        return x

my_seed = 42
seed_assign(my_seed)

# 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 준비
train_dataset = Qurioga_Spike(root='/data2/spike_sorting/quiroga/', train=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = Qurioga_Spike(root='/data2/spike_sorting/quiroga/', train=False)
test_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

# 모델, 손실 함수, 최적화 기법 정의
model = SimpleDNN().to(device)
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류에 적합한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 최적화

best_test_accuracy = 0

# 학습 루프
num_epochs = 1000  # 학습 에폭 수

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 전환
    running_loss = 0.0
    correct = 0
    total = 0

    for data, labels in train_loader:
        # 데이터를 GPU로 이동 (가능하면)
        data, labels = data.to(device), labels.to(device)

        # Optimizer 기울기 초기화
        optimizer.zero_grad()

        # 순전파
        outputs = model(data)
        
        # 손실 계산
        loss = criterion(outputs, labels)
        
        # 역전파
        loss.backward()
        
        # 가중치 업데이트
        optimizer.step()

        running_loss += loss.item()

        # 예측
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    # 테스트 단계
    model.eval()  # 모델을 평가 모드로 전환
    test_correct = 0
    test_total = 0

    with torch.no_grad():  # 그래디언트 비활성화
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            
            # 순전파
            outputs = model(data)

            # 예측
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    epoch_train_loss = running_loss / len(train_loader)
    epoch__train_accuracy = 100 * correct / total
    epoch_test_accuracy = 100 * test_correct / test_total

    best_test_accuracy = max(epoch_test_accuracy, best_test_accuracy)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch__train_accuracy:.2f}%, Test Accuracy: {epoch_test_accuracy:.2f}%, Best Test Accuracy: {best_test_accuracy:.2f}%')

print('Finished Training')


Epoch [1/1000], Train Loss: 0.7534, Train Accuracy: 75.01%, Test Accuracy: 82.49%, Best Test Accuracy: 82.49%
Epoch [2/1000], Train Loss: 0.4345, Train Accuracy: 83.22%, Test Accuracy: 84.60%, Best Test Accuracy: 84.60%
